In [52]:
pip install streamlit pandas numpy scikit-learn

In [53]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay
from sklearn.metrics import precision_score, recall_score

st.set_option('deprecation.showPyplotGlobalUse', False)

def main():
    st.title("Binary Classification Web App")
    st.sidebar.title("Classification")
    st.markdown("Are your mushrooms edible or poisonous ? 🍄🍄🍄")
    st.sidebar.markdown("-----🍄🍄🍄-----")

    @st.cache_data(persist=True)
    def load_data():
        data = pd.read_csv('/primary_data.csv', delimiter= ";")
        label = LabelEncoder()
        imp = SimpleImputer(strategy="most_frequent")
        imp.fit_transform(df)
        for col in data.columns:
            data[col] = label.fit_transform(data[col])
        return data

    @st.cache_data(persist=True)
    def split(df):
        y= df["class"]
        x= df.drop(df.columns[0], axis=1)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
        return x_train, x_test, y_train, y_test

    def plot_metrics(metrics_list):
        if 'Confusion Matrix' in metrics_list:
            st.subheader("Confusion Matrix")
            ConfusionMatrixDisplay.from_estimator(model, x_test, y_test, display_labels=class_names)
            st.pyplot()

        if 'ROC Curve' in metrics_list:
            st.subheader("ROC Curve")
            RocCurveDisplay.from_estimator(model, x_test, y_test)
            st.pyplot()

        if 'Precision-Recall Curve' in metrics_list:
            st.subheader("Precision-Recall Curve")
            PrecisionRecallDisplay.from_estimator(model, x_test, y_test)
            st.pyplot()


    df = load_data()
    x_train, x_test, y_train, y_test = split(df)
    x_train = x_train.copy()
    y_train = y_train.copy()
    x_test = x_train.copy()
    y_test = y_train.copy()


    class_names = ['edible', 'poisonous']
    st.sidebar.subheader("Choose a Classifier")
    classifier = st.sidebar.selectbox("Classifier", ("Support Vector Machine (SVM)", "Logistic Regression", "Random Forest"))

    if classifier == 'Support Vector Machine (SVM)':
        st.sidebar.subheader("Model Hyperparameters")
        C = st.sidebar.number_input("C (Regularization Parameter)", 0.01, 10.0, step=0.01, key='C')
        kernel = st.sidebar.radio("Kernel", ("rbf", "linear"), key='kernel')
        gamma = st.sidebar.radio("Gamma (Kernel Coefficient)", ("scale", "auto"), key='gamma')

        metrics = st.sidebar.multiselect("What metrics to plot ?", ('Confusion Matrix', 'ROC Curve', 'Precision-Recall Curve'))

        if st.sidebar.button("Classify", key='classify'):
            st.subheader('Support Vector Machine (SVM) Results')
            model = SVC(C=C, kernel=kernel, gamma=gamma)
            model.fit(x_train, y_train)
            y_pred = model.predict(x_test)
            accuracy = model.score(x_test, y_test)
            precision = precision_score(y_test, y_pred, labels=class_names)
            recall = recall_score(y_test, y_pred, labels=class_names)

            rounded_accuracy = round(accuracy, 2)
            st.write("Accuracy: ", rounded_accuracy)
            rounded_pecision = round(precision, 2)
            st.write("Precision: ", rounded_pecision)
            rounded_recall = round(recall, 2)
            st.write("Recall: ", rounded_recall)
            plot_metrics(metrics)

    if classifier == 'Logistic Regression':
        st.sidebar.subheader("Model Hyperparameters")
        C = st.sidebar.number_input("C (Regularization Parameter)", 0.01, 10.0, step=0.01, key='C_LR')
        max_iter = st.sidebar.slider("Maximum number of iterations", 100, 500, key='max_iter')

        metrics = st.sidebar.multiselect("What metrics to plot ?", ('Confusion Matrix', 'ROC Curve', 'Precision-Recall Curve'))

        if st.sidebar.button("Classify", key='classify'):
            st.subheader('Logistic Regression Results')
            model = LogisticRegression(C=C, max_iter=max_iter)
            model.fit(x_train, y_train)
            accuracy = model.score(x_test, y_test)
            y_pred = model.predict(x_test)
            accuracy = model.score(x_test, y_test)
            precision = precision_score(y_test, y_pred, labels=class_names)
            recall = recall_score(y_test, y_pred, labels=class_names)

            rounded_accuracy = round(accuracy, 2)
            st.write("Accuracy: ", rounded_accuracy)
            rounded_pecision = round(precision, 2)
            st.write("Precision: ", rounded_pecision)
            rounded_recall = round(recall, 2)
            st.write("Recall: ", rounded_recall)
            plot_metrics(metrics)


    if classifier == 'Random Forest':
        st.sidebar.subheader("Model Hyperparameters")
        n_estimators = st.sidebar.number_input("The number of trees in the forest", 100, 5000, step=10, key='n_estimators')
        max_depth = st.sidebar.number_input("The maximum depth of the tree", 1, 20, step=1, key='max_depth')
        bootstrap = st.sidebar.radio("Bootsrap samples when building trees", (True, False), key='bootstrap')


        metrics = st.sidebar.multiselect("What metrics to plot ?", ('Confusion Matrix', 'ROC Curve', 'Precision-Recall Curve'))

        if st.sidebar.button("Classify", key='classify'):
            st.subheader('Random Forest Results')
            model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, bootstrap=bootstrap, n_jobs=-1)
            model.fit(x_train, y_train)
            accuracy = model.score(x_test, y_test)
            y_pred = model.predict(x_test)
            accuracy = model.score(x_test, y_test)
            precision = precision_score(y_test, y_pred, labels=class_names)
            recall = recall_score(y_test, y_pred, labels=class_names)

            rounded_accuracy = round(accuracy, 2)
            st.write("Accuracy: ", rounded_accuracy)
            rounded_pecision = round(precision, 2)
            st.write("Precision: ", rounded_pecision)
            rounded_recall = round(recall, 2)
            st.write("Recall: ", rounded_recall)
            plot_metrics(metrics)











    if st.sidebar.checkbox("Show raw data", False):
        st.subheader("Mushroom Dataset Information")
        st.markdown("This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family (pp. 500-525).  Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended.  This latter class was combined with the poisonous one.  The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like ``leaflets three, let it be'' for Poisonous Oak and Ivy.")
        st.subheader("Has Missing Values ?")
        st.markdown("Yes")
        st.subheader("Preview on the mushrooms dataset")
        st.write(df)





if __name__ == '__main__':
    main()




Overwriting app.py


In [54]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.928s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [55]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.194.141.115


In [56]:
!streamlit run app.py &>/content/logs.txt &

In [57]:
!npx localtunnel --port 8501

npx: installed 22 in 3.019s
your url is: https://tidy-socks-pump.loca.lt
^C
